In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.webgui import Draw

### Permanentmagnet
$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\R}{\mathbb{R}}
$$
Finde $ u \in H(\Curl)$ sodass 
$$
\int \mu^{-1} \Curl u \Curl v + 10^{-8} \int  u  v= \int M \Curl v \qquad  \forall \, v \in H(\Curl) \tag{1}
$$


Wobei der zweite Integralterm der Stabilität dient und die Werte für $\mu_r$ wie folgt angenommen werden:
* magnet: $\mu_r = 1$
* copper plate: $\mu_r = 1-6.4\cdot10^{-6}$
* iron plate: $\mu_r = 1000$
* air: $\mu_r = 1$ 


In [2]:
geo = CSGeometry()
box_size = 5
box = OrthoBrick(Pnt(-box_size,-box_size,-box_size), Pnt(box_size,box_size,box_size)).bc("outer")
magnet = Cylinder(Pnt(0.05,0,0), Pnt(0.05,0,1), 0.3) * \
            OrthoBrick (Pnt(-1,-1,-1),Pnt(1,1,1)).mat("magnet")

plate = OrthoBrick (Pnt(-1, 1,-1.5),Pnt(1,1.2,1.5)).mat('copper')
# plate = OrthoBrick (Pnt(-1, 1,-1.5),Pnt(1,1.2,1.5)).mat('iron')

air = (box - magnet- plate).mat('air') 

geo.Add(plate)#, col=(0,1,0)) # what is col?
geo.Add(magnet)
geo.Add(air)#, transparent=True)

mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(5)
Draw (mesh)
# help (geo)
# help(box)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-9-ga349a5c89', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, …

In [3]:
fes = HCurl(mesh, order = 3, dirichlet="outer", nograds=True)
u,v = fes.TnT()
from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1 if mat== "magnet" else 1-6.4*1e-7 if mat=='copper' 1000 if mat = 'iron' else 1
                            for mat in mesh.GetMaterials()]) 
a = BilinearForm(fes)
a += 1/(mu0*mur)*curl(u)*curl(v)*dx + 1e-8/(mu0*mur)*u*v*dx
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((0,0,1)) * \
    CoefficientFunction( [100 if mat == "magnet" else 0 for mat in mesh.GetMaterials()])
f += mag*curl(v)*dx(mesh.Materials("magnet"))

gfu = GridFunction(fes)

with TaskManager():
    a.Assemble()
    f.Assemble()
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat)

# help(c)
# help(fes)

SyntaxError: invalid syntax (<ipython-input-3-7efb50609064>, line 5)

In [ ]:
## magnetic flux
Draw (curl(gfu), mesh, "B-field", draw_surf=False)

In [ ]:
Draw (gfu, mesh, "vector-potential", draw_surf = False)

In [ ]:
## magnetic field
Draw (1/(mu0*mur)*curl(gfu)-mag, mesh, "H-field", draw_surf=False)

## Notizen für mich
nützliche links:

* https://de.wikipedia.org/wiki/Maxwell-Gleichungen
* https://ngsolve.org/docu/latest/i-tutorials/wta/maxwell.html
* https://ngsolve.org/docu/latest/i-tutorials/unit-2.3-hcurlhdiv/hcurlhdiv.html
* https://ngsolve.org/docu/latest/i-tutorials/unit-2.4-Maxwell/Maxwell.html
* https://www.asc.tuwien.ac.at/~schoeberl/wiki/lva/notes/maxwell.pdf
* https://www.numa.uni-linz.ac.at/Teaching/Bachelor/kagerer-bakk.pdf
* "allwissender Kreis" 

### Fragen

* Ist es möglich die äußere Box unsichtbar zu machen?
* Was ist `col` bei geo.Add(..)
* Was bringt `nograds` (lt Help 'Remove higher order gradients of H1 basis functions from HCurl FESpace')?
* mehr clipping planes? 
* impressed current density?

### Basics

#### Physikalische Größen

$E$ .. Elektrische Feldstärke (Stärke und Richtung eines elektrischen Feldes) \
$D$ .. Elektrische Flussdichte (Dichte der elektrischen Feldlinien in Bezug auf eine Fläche) \
$H$ .. Magnetische Felstärke \
$B$ .. Magnetische Flussdichte \
$\rho$ .. Ladungsdichte (Ladung pro Volumen) \
$\sigma$ .. elektrische Leitfähigkeit \
$j$ .. Stromdichte (Strom pro durchflossene Fläche) 


#### Makroskopische Maxwelllgleichungen
(M1) $\Div D = \rho$ (Ladung ist Quelle des elektrischen Feldes) \
(M2) $\Div B = 0$ (keine magnetischen Ladungen) \
(M3) $\Curl E = -\partial_t B$ (Induktion)  \
(M4) $\Curl H = j + \partial_t D$ (Amper'sches Gesetz)

#### sonstige Zusammenhänge

(S1) $D = \epsilon E $ (evtl noch $+ P$ .. Polarisation, was ist das? vernachlässigbar für unsere Anwendungen?) wobei $\epsilon$ .. Dielektrizitätskonstante \
(S2) $H = \mu^{-1} B - M$ wobei $\mu = \mu_0 \mu_r$ (magnetische Permeabilität), $\mu_0 =  4\pi10^{-7}$ und $M$ die Magnetisierung ist (wenn im linearen Bereich) \
(S3) $j = \sigma E +j_i$ ($j_i$ .. 'impressed current density', was ist das genau?)


#### Herleitung einer schwachen Formulierung
Einführen eines Vektorpotentials $A$ sodass $B = \Curl A$. 

Umformen:
$$
\begin{split}
\Curl(\mu^{-1} \Curl(A) - M) &= \Curl(\mu^{-1} B - M) \\
&\stackrel{\text{(S2)}}{=} \Curl(H) \\
&\stackrel{\text{(M4)}}{=} j + \partial_t D \\
&\stackrel{\text{(S1)}}{=} j + \partial_t \epsilon E  \\
&\stackrel{\text{(S3)}}{=} \sigma E + j_i + \partial_t \epsilon E 
\end{split}
$$


Ziel ist $A$ so zu wählen, dass $E = -\partial_t A$. Einzige Bedingung an $A$ war, dass $\Curl A = B$, dh jedes $\tilde{A}$ das sich von $A$ nur um ein Gradientenfeld unterscheidet erfüllt das genauso. Sei $\tilde{A}$ so dass $\Curl \tilde{A} = B$



Nach (M3) gilt $\Curl E = - \partial_t (\Curl \tilde{A})) = - \Curl( \partial_t \tilde{A})$. Äquivalent dazu $\Curl(E + \partial_t \tilde{A}) = 0$. Gibt Potential $\phi: \R^3 \rightarrow \R $ mit $\Grad \phi =  E+ \partial_t \tilde{A}$ wenn das Gebiet brav genug ist (sternförmig oder einfach zusammenhängend). Angenommen das ist der Fall dann gilt $E = \Grad \phi - \partial_t \tilde{A}$. Mit 
$$
A = - (\int_{t_0}^{t} \Grad \phi(s) ~ \text{ds} - \tilde{A})
$$ 
ist die gewünschte Anforderung erfüllt. Dh wir können annehmen $E = -\partial_t A$. 

Einsetzen ergibt
$$
\Curl(\mu^{-1} \Curl(A) - M) + \sigma \partial_t A +  \epsilon \partial_t^2A = j_i .
$$


Partielle Integration für curl 
$$
\int_\Omega \Curl u \cdot v = \int_\Omega u \cdot \Curl v - \int_{\partial\Omega} (u \times n) \cdot v
$$


Angenommen wir haben Dirichlet RB, dann ergibt sich die schwache Formulierung:

Finde $A \in H(\Curl)$ sodass 
$$ 
\int \mu^{-1} \Curl A \Curl v +  \sigma \int \partial_t A v + \epsilon \int \partial_t^2 A v = \int j_i v + \int M \Curl v \qquad \forall v \in H(\Curl). \tag{*}
$$

#### Spezialfälle

Permanentmagnet:  $j_i = 0$ und  $E = 0$ $\Rightarrow$ (1)

